In [3]:
import os
import pickle
import numpy as np
import math
import random
import tqdm
from collections import defaultdict
from scipy.spatial import distance_matrix
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## Implement the simulator of graph stream

#### The stochastic block model class

In [4]:
class SBMGraphStream():
    '''
    The class of Graph Stream from the stochastic block model
    ----- Parameters -----
    # n_vertex: the number of vertices in the graph
    # p_intra: the probability for + edge (u,v) for the same cluster
    # p_inter: the probability for + edge (u,v) for different clusters
    # k_cluster: number of clusters in the clustering
    ----- Methods ----
    # read_next_edge(): read the next edge and move the index +1
    ----- Representation ----
    The graph is representation with an indexed array of vertices and a dictionary with (u_i, u_j): labels
    '''
    
    def __init__(self, n_vertex, p_intra=0.8, p_inter=0.2, k_cluster=7):
        '''
        :param n_vertex: the the number of vertices in the graph
        '''
        self.n_vertex = n_vertex
        self.p_intra = p_intra
        self.p_inter = p_inter
        self.k_cluster = k_cluster
        
        # initialize the vertex set and the cluster labels
        self.vertex_set = np.array([self.n_vertex])
        num_v_per_cluster = n_vertex//self.k_cluster
        n_residual = n_vertex % num_v_per_cluster
        cluster_labels_list = []
        for i_cluster in range(k_cluster):
            cluster_labels_list.append(i_cluster*np.ones([num_v_per_cluster]))
        if n_residual!=0:
            cluster_labels_list.append((k_cluster-1)*np.ones([n_residual]))
        # collect them as a 1-d array
        self.cluster_labels = np.reshape(np.hstack(cluster_labels_list).astype(int), [-1])
        # initialize the edges -- using +1 and -1 to represent the edge labels
        # also compute the cost
        self.cc_cost = 0
        self.edge_dict = {}
        for u_i in tqdm.tqdm(range(self.n_vertex)):
            for u_j in np.arange(u_i+1, self.n_vertex):
                if self.cluster_labels[u_i] == self.cluster_labels[u_j]:
                    if np.random.rand() <= p_intra:
                        self.edge_dict[(u_i,u_j)] = 1
                    else:
                        self.edge_dict[(u_i,u_j)] = -1
                        self.cc_cost = self.cc_cost + 1
                else:
                    if np.random.rand() <= p_inter:
                        self.edge_dict[(u_i,u_j)] = 1
                        self.cc_cost = self.cc_cost + 1
                    else:
                        self.edge_dict[(u_i,u_j)] = -1
        # randomize the order of edge arrival
        self.edge_names = list(self.edge_dict.keys())
        random.shuffle(self.edge_names)
        self.num_edges = len(self.edge_names)
        # maintain a pointer of the number of edges
        self.current_stream_ind = 0
        
    def read_next_edge(self):
        
        this_edge_name = self.edge_names[self.current_stream_ind]
        this_edge_label = self.edge_dict[this_edge_name]
        self.current_stream_ind = self.current_stream_ind + 1
        if self.current_stream_ind>=self.num_edges-1:
            return None, None
        
        return this_edge_name, this_edge_label
    
    def reset_index(self):
        '''
        reset the pointer
        '''
        self.current_stream_ind = 0

In [5]:
sbm_graph_stream = SBMGraphStream(n_vertex=1000)

100%|██████████| 1000/1000 [00:00<00:00, 2117.07it/s]


In [14]:
print(sbm_graph_stream.num_edges)
first_edge, first_edge_label = sbm_graph_stream.read_next_edge()
print('The first edge is ', first_edge, 'and the label is ', first_edge_label)
sbm_graph_stream.reset_index()

499500
The first edge is  (329, 648) and the label is  1


#### The Erdos-Renyi graph class 

In [15]:
class ErdosRenyiGraphStream():
    '''
    The class of Graph Stream from the Erdos-Renyi graph
    ----- Parameters -----
    # n_vertex: the number of vertices in the graph
    # p_edge: the probability for an edge to be sampled
    ----- Methods ----
    # read_next_edge(): read the next edge and move the index +1
    ----- Representation ----
    The graph is representation with an indexed array of vertices and a dictionary with (u_i, u_j): labels
    '''
    
    def __init__(self, n_vertex, p_edge=0.5):
        '''
        :param n_vertex: the the number of vertices in the graph
        '''
        self.n_vertex = n_vertex
        self.p_edge = p_edge
        
        # initialize the vertex set and the cluster labels
        self.vertex_set = np.array([self.n_vertex])
        # maintain the cost as (basically) the number of (+) edges
        self.cc_cost = 0
        self.edge_dict = {}
        for u_i in tqdm.tqdm(range(self.n_vertex)):
            for u_j in np.arange(u_i+1, self.n_vertex):
                if np.random.rand() <= self.p_edge:
                    self.edge_dict[(u_i,u_j)] = 1
                    self.cc_cost = self.cc_cost + 1
                else:
                    self.edge_dict[(u_i,u_j)] = -1
        # randomize the order of edge arrival
        self.edge_names = list(self.edge_dict.keys())
        random.shuffle(self.edge_names)
        self.num_edges = len(self.edge_names)
        # maintain a pointer of the number of edges
        self.current_stream_ind = 0
        
    def read_next_edge(self):
        
        this_edge_name = self.edge_names[self.current_stream_ind]
        this_edge_label = self.edge_dict[this_edge_name]
        self.current_stream_ind = self.current_stream_ind + 1
        if self.current_stream_ind>=self.num_edges-1:
            return None, None
        
        return this_edge_name, this_edge_label
    
    def reset_index(self):
        '''
        reset the pointer
        '''
        self.current_stream_ind = 0

In [16]:
random_graph_stream = ErdosRenyiGraphStream(n_vertex=1000)

100%|██████████| 1000/1000 [00:00<00:00, 2028.17it/s]


In [17]:
print(random_graph_stream.num_edges)
first_edge, first_edge_label = random_graph_stream.read_next_edge()
print('The first edge is ', first_edge, 'and the label is ', first_edge_label)
random_graph_stream.reset_index()

499500
The first edge is  (810, 813) and the label is  1


In [18]:
print('The optimal SBM cost is ', sbm_graph_stream.cc_cost)
print('The optimal random graph cost is ', random_graph_stream.cc_cost)

The optimal SBM cost is  100041
The optimal random graph cost is  249479


## Implementation of the SDD-based algorithms

In [19]:
class SparseEdgeTool:
    '''
    A class that tests if a given edge is sparse (on the `+' edges)
    '''
    
    def __init__(self, edge_name, n_vertex, eps=0.2):
        '''
        :param n_vertex: the the number of vertices in the graph
        '''
        self.edge_name = edge_name
        self.n_vertex = n_vertex
        self.eps = eps
        # sample a set of log n vertices
        self.n_sample_vertex = max((int)(5*np.log(self.n_vertex)), 20)
        self.sample_vertex_set = np.random.choice(self.n_vertex, self.n_sample_vertex, replace=False)
        self.endpoint_one = edge_name[0]
        self.endpoint_two = edge_name[1]
        self.neighbors_one = []
        self.neighbors_two = []
        self.degree_one = 0
        self.degree_two = 0
        self.plabel = False
        
    def read_edge_and_process(self, e, label):
        if label == 1:
            # count the degree
            if (e[0]==self.endpoint_one) or (e[1]==self.endpoint_one):
                self.degree_one += 1
            if (e[0]==self.endpoint_two) or (e[1]==self.endpoint_two):
                self.degree_two += 1
            # determine the sampled neightbors
            if (e[0] in self.sample_vertex_set) or (e[1] in self.sample_vertex_set):
                # process the edge if connected to the first endpoint
                if (e[0]==self.endpoint_one) or (e[1]==self.endpoint_one):
                    arrive_edge_list = list(e)
                    arrive_edge_list.remove(self.endpoint_one)
                    self.neighbors_one.append(arrive_edge_list[0])
                # process the edge if connected to the second endpoint
                if (e[0]==self.endpoint_two) or (e[1]==self.endpoint_two):
                    arrive_edge_list = list(e)
                    arrive_edge_list.remove(self.endpoint_two)
                    self.neighbors_two.append(arrive_edge_list[0])
            elif (e[0] in self.edge_name) and (e[1] in self.edge_name):
                self.plabel = True
        else:
            pass
                
                
    def determine_sparse(self):
        dif1 = np.setdiff1d(self.neighbors_one, self.neighbors_two)
        dif2 = np.setdiff1d(self.neighbors_two, self.neighbors_one)
        total_diff = np.concatenate([dif1, dif2])
        total_elements = list(set(self.neighbors_one + self.neighbors_two))
        
        if (len(total_diff)>=self.eps*len(total_elements)) and self.plabel:
            return 1
        else:
            return 0

In [20]:
class DenseEdgeTool:
    '''
    A class that tests if a given edge is dense (on the `--' edges)
    '''
    
    def __init__(self, edge_name, n_vertex, eps=0.2):
        '''
        :param n_vertex: the the number of vertices in the graph
        '''
        self.edge_name = edge_name
        self.n_vertex = n_vertex
        self.eps = eps
        # sample a set of log n vertices
        self.n_sample_vertex = max((int)(5*np.log(self.n_vertex)), 20)
        self.sample_vertex_set = np.random.choice(self.n_vertex, self.n_sample_vertex, replace=False)
        self.endpoint_one = edge_name[0]
        self.endpoint_two = edge_name[1]
        self.degree_one = 0
        self.degree_two = 0
        self.neighbors_one = []
        self.neighbors_two = []
        self.nlabel = False
        
    def read_edge_and_process(self, e, label):
        if label == 1:
            # count the degree
            if (e[0]==self.endpoint_one) or (e[1]==self.endpoint_one):
                self.degree_one += 1
            if (e[0]==self.endpoint_two) or (e[1]==self.endpoint_two):
                self.degree_two += 1
            # determine the sampled neighbors
            if (e[0] in self.sample_vertex_set) or (e[1] in self.sample_vertex_set):
                # process the edge if connected to the first endpoint
                if (e[0]==self.endpoint_one) or (e[1]==self.endpoint_one):
                    arrive_edge_list = list(e)
                    arrive_edge_list.remove(self.endpoint_one)
                    self.neighbors_one.append(arrive_edge_list[0])
                # process the edge if connected to the second endpoint
                if (e[0]==self.endpoint_two) or (e[1]==self.endpoint_two):
                    arrive_edge_list = list(e)
                    arrive_edge_list.remove(self.endpoint_two)
                    self.neighbors_two.append(arrive_edge_list[0])
        elif (e[0] in self.edge_name) and (e[1] in self.edge_name):
            self.nlabel = True
        
    def determine_dense(self):
        dif1 = np.setdiff1d(self.neighbors_one, self.neighbors_two)
        dif2 = np.setdiff1d(self.neighbors_two, self.neighbors_one)
        total_diff = np.concatenate([dif1, dif2])
        total_elements = list(set(self.neighbors_one + self.neighbors_two))
        
        if (len(total_diff)<self.eps*len(total_elements)) and self.nlabel:
            return 1
        else:
            return 0

In [21]:
# the main SDD-based streaming value tester
def SDD_cost_eval(graph_stream, eps=0.2, delta=0.1, add_error=False, rescale_spr=False):
    # parameters
    n_vertex = graph_stream.n_vertex
    n_edges = sbm_graph_stream.num_edges
    n_sparse_pairs = max((int)(5*np.log(n_vertex)), 20)
    n_dense_vertex = max((int)(5*np.log(n_vertex)), 15)
    n_dense_vertex_neighbors =  max((int)(5*np.log(n_vertex)), 15)
    total_edges_store = 0
    
    # initialize all the tools for process
    print('Pre-processing...')
    sparse_edge_tools_list = [] # this will be a list of tools
    dense_vertex_tools_list = [] # this will be a list of lists of tools
    for c_tool in range(n_sparse_pairs):
        this_edge = tuple(np.random.choice(n_vertex, 2, replace=False))
        sparse_edge_tools_list.append(SparseEdgeTool(this_edge, n_vertex, eps=eps))
    for c_vertex in range(n_dense_vertex):
        # main n_dense_vertex_neighbors tools for each of the vertex
        base_vertex = np.random.choice(n_vertex, 1)[0]
        this_vertex_dense_tool_list = []
        for c_neighbor in range(n_dense_vertex_neighbors):
            # repeat sample -- could run to infinite lool but very very unlikely
            while(True):
                this_neighbor_vertex = np.random.choice(n_vertex, 1)[0]
                if this_neighbor_vertex!= base_vertex:
                    break
            # creat a tool and append
            this_vertex_dense_tool_list.append(DenseEdgeTool((base_vertex, this_neighbor_vertex), n_vertex, eps=eps))
        dense_vertex_tools_list.append(this_vertex_dense_tool_list)
    
    print('Processing the stream...')
    # read the stream and process on-the-fly
    for i in tqdm.tqdm(range(n_edges)):
        this_edge, this_edge_label = sbm_graph_stream.read_next_edge()
        if this_edge is None:
            break
        for c_tool in range(n_sparse_pairs):
            sparse_edge_tools_list[c_tool].read_edge_and_process(this_edge, this_edge_label)
        for c_vertex in range(n_dense_vertex):
            for c_neighbor in range(n_dense_vertex_neighbors):
                dense_vertex_tools_list[c_vertex][c_neighbor].read_edge_and_process(this_edge, this_edge_label)
                
    # post-processing
    print('Post-processing...')
    # estimation of E-spr
    E_spr_est = 0
    valid_E_spr = 0
    for c_tool in range(n_sparse_pairs): 
        E_spr_est += sparse_edge_tools_list[c_tool].determine_sparse()
        total_edges_store = total_edges_store + len(sparse_edge_tools_list[c_tool].
                                                    neighbors_one) + len(sparse_edge_tools_list[c_tool].neighbors_two)
        if (sparse_edge_tools_list[c_tool].degree_one>=
            delta*n_vertex) and (sparse_edge_tools_list[c_tool].degree_two>=delta*n_vertex):
            valid_E_spr += 1
    E_spr_est_final = E_spr_est*(n_edges/valid_E_spr)
    # estimation of E-dens
    E_den_est = 0
    for c_vertex in range(n_dense_vertex):
        this_base_vertex_degree = dense_vertex_tools_list[c_vertex][0].degree_one
        for c_neighbor in range(n_dense_vertex_neighbors):
            total_edges_store = total_edges_store + len(
                dense_vertex_tools_list[c_vertex][c_neighbor].neighbors_one) + len(
                dense_vertex_tools_list[c_vertex][c_neighbor].neighbors_two)
        if this_base_vertex_degree<delta*n_vertex:
            continue
        this_vertex_est = 0
        for c_neighbor in range(n_dense_vertex_neighbors):
            this_vertex_est += dense_vertex_tools_list[c_vertex][c_neighbor].determine_dense()
        Y_v = min(this_base_vertex_degree, this_vertex_est*(n_vertex/n_dense_vertex_neighbors))
        E_den_est += Y_v
    # putting everything together
    if add_error:
        E_spr_est_final = E_spr_est_final + delta*n_edges
        E_den_est = E_den_est + delta*n_edges
    if rescale_spr:
        E_spr_est_final = E_spr_est_final/eps
    
    return E_den_est + E_spr_est_final, total_edges_store

In [22]:
sbm_graph_stream.reset_index()
SDD_cost_est, SDD_total_stored_edge = SDD_cost_eval(sbm_graph_stream)

Pre-processing...


  0%|          | 24/499500 [00:00<34:42, 239.89it/s]

Processing the stream...


100%|█████████▉| 499498/499500 [22:41<00:00, 366.98it/s] 


Post-processing...


In [23]:
print('The expected SBM cost is ', sbm_graph_stream.cc_cost)
print('The estimated correlation clustering cost is ', SDD_cost_est)

The expected SBM cost is  100041
The estimated correlation clustering cost is  44073.529411764706


In [24]:
print('The number of edges in the graphs is ', sbm_graph_stream.num_edges)
print('The number of edges stored by the algorithm is ', SDD_total_stored_edge)

The number of edges in the graphs is  499500
The number of edges stored by the algorithm is  44068


## Implementation of the Pivot-based algorithm
Barrier: it seems it's hard for us to actually go through 2^(1/delta) permutations.

In [25]:
tuple_1 = (1,20)
tuple_2 = (20, 19, 98, 71)
aa = list(set(tuple_1).intersection(set(tuple_2)))

In [26]:
class PredAwareNE:
    '''
    This is the class that implements the predecessor-aware non-edge sketh
    '''
    def __init__(self, vertex_name, n_vertex, pred_set, eps=0.2, delta=0.1):
        '''
        :param n_vertex: the the number of vertices in the graph
        '''
        if vertex_name in pred_set:
            raise ValueError('The tested vertex cannot be inside the set!')
        self.n_vertex = n_vertex
        self.n_edges = (int)(n_vertex*(n_vertex-1)/2)
        self.vertex_name = vertex_name
        self.vertex_degree = 0
        self.eps = eps
        self.delta = delta
        # sample a set of log n vertices
        self.n_sample_edges = max((int)(5*np.log(self.n_vertex)), 20)
        self.sample_edge_set = []
        self.first_endpoint_set = []
        self.second_endpoint_set = []
        # keep track of the endpoints' connection to u and S
        self.first_endpoint_to_u = self.n_sample_edges*[False]
        self.first_endpoint_to_S = self.n_sample_edges*[False]
        self.second_endpoint_to_u = self.n_sample_edges*[False]
        self.second_endpoint_to_S = self.n_sample_edges*[False]
        for i_edge in range(self.n_sample_edges):
            # to avoid sample the same vertex, try repetitive sampling
            while(True):
                first_vertex = np.random.choice(self.n_vertex, 1)[0]
                second_vertex = np.random.choice(self.n_vertex, 1)[0]
                if first_vertex!=second_vertex:
                    break
            self.sample_edge_set.append((first_vertex, second_vertex))
            self.first_endpoint_set.append(first_vertex)
            self.second_endpoint_set.append(second_vertex)
            if first_vertex == self.vertex_name:
                self.first_endpoint_to_u[i_edge] = True
            if second_vertex == self.vertex_name:
                self.second_endpoint_to_u[i_edge] = True
            if first_vertex in pred_set:
                self.first_endpoint_to_S[i_edge] = True
            if second_vertex in pred_set:
                self.second_endpoint_to_S[i_edge] = True
        # edge labels
        self.edge_label = self.n_sample_edges*[0]
        self.pred_set = pred_set
        
    def read_edge_and_process(self, e, label):
        for i_edge in range(self.n_sample_edges):
            # if edge is some (x,y), update the label
            this_edge_name = self.sample_edge_set[i_edge]
            if this_edge_name==e or this_edge_name==(e[1],e[0]):
                self.edge_label[i_edge] = label
            # if edge connects x (or y) to u, mark it
            if (self.first_endpoint_set[i_edge] in e) and (self.vertex_name in e) and (label == 1):
                self.first_endpoint_to_u[i_edge] = True
            if (self.second_endpoint_set[i_edge] in e) and (self.vertex_name in e) and (label == 1):
                self.second_endpoint_to_u[i_edge] = True
            # if there is intersection with S, mark it
            if (self.first_endpoint_set[i_edge] in e) and list(set(e).
                                                               intersection(set(self.pred_set))) and (label == 1):
                self.first_endpoint_to_S[i_edge] = True
            if (self.second_endpoint_set[i_edge] in e) and list(set(e).intersection(set(self.pred_set))) and (label == 1):
                self.second_endpoint_to_S[i_edge] = True
        # don't forget to count degree :D
        if self.vertex_name in e and label==1:
            self.vertex_degree += 1
                
    def determine_cost(self):
        inside_non_edge_cost = 0
        if self.vertex_degree<=self.delta*self.n_vertex:
            return self.delta*self.n_vertex
        else:
            for i_edge in range(self.n_sample_edges):
                if (self.first_endpoint_to_u[i_edge]
                   ) and (self.second_endpoint_to_u[i_edge]
                         ) and (not self.first_endpoint_to_S[i_edge]
                               ) and (not self.second_endpoint_to_S[i_edge]
                                     ) and (self.edge_label[i_edge]==-1):
                    inside_non_edge_cost +=1
            
            return inside_non_edge_cost*self.n_edges/self.n_sample_edges

In [27]:
class PredAwareE:
    '''
    This is the class that implements the predecessor-aware non-edge sketh
    '''
    def __init__(self, vertex_name, n_vertex, pred_set, eps=0.2, delta=0.1):
        '''
        :param n_vertex: the the number of vertices in the graph
        '''
        if vertex_name in pred_set:
            raise ValueError('The tested vertex cannot be inside the set!')
        self.n_vertex = n_vertex
        self.n_edges = (int)(n_vertex*(n_vertex-1)/2)
        self.vertex_name = vertex_name
        self.vertex_degree = 0
        self.eps = eps
        self.delta = delta
        # sample a set of log n vertices
        self.n_sample_edges = max((int)(5*np.log(self.n_vertex)), 20)
        self.sample_edge_set = []
        self.first_endpoint_set = []
        self.second_endpoint_set = []
        # keep track of the endpoints' connection to u and S
        self.first_endpoint_to_u = self.n_sample_edges*[False]
        self.first_endpoint_to_S = self.n_sample_edges*[False]
        self.second_endpoint_to_u = self.n_sample_edges*[False]
        self.second_endpoint_to_S = self.n_sample_edges*[False]
        for i_edge in range(self.n_sample_edges):
            # to avoid sample the same vertex, try repetitive sampling
            while(True):
                first_vertex = np.random.choice(self.n_vertex, 1)[0]
                second_vertex = np.random.choice(self.n_vertex, 1)[0]
                if first_vertex!=second_vertex:
                    break
            self.sample_edge_set.append((first_vertex, second_vertex))
            self.first_endpoint_set.append(first_vertex)
            self.second_endpoint_set.append(second_vertex)
            if first_vertex == self.vertex_name:
                self.first_endpoint_to_u[i_edge] = True
            if second_vertex == self.vertex_name:
                self.second_endpoint_to_u[i_edge] = True
            if first_vertex in pred_set:
                self.first_endpoint_to_S[i_edge] = True
            if second_vertex in pred_set:
                self.second_endpoint_to_S[i_edge] = True
        # edge labels
        self.edge_label = self.n_sample_edges*[0]
        self.pred_set = pred_set
        
    def read_edge_and_process(self, e, label):
        for i_edge in range(self.n_sample_edges):
            # if edge is some (x,y), update the label
            this_edge_name = self.sample_edge_set[i_edge]
            if this_edge_name==e or this_edge_name==(e[1],e[0]):
                self.edge_label[i_edge] = label
            # if edge connects x (or y) to u, mark it
            if (self.first_endpoint_set[i_edge] in e) and (self.vertex_name in e) and (label == 1):
                self.first_endpoint_to_u[i_edge] = True
            if (self.second_endpoint_set[i_edge] in e) and (self.vertex_name in e) and (label == 1):
                self.second_endpoint_to_u[i_edge] = True
            # if there is intersection with S, mark it
            if (self.first_endpoint_set[i_edge] in e) and list(set(e).
                                                               intersection(set(self.pred_set))) and (label == 1):
                self.first_endpoint_to_S[i_edge] = True
            if (self.second_endpoint_set[i_edge] in e) and list(set(e).intersection(set(self.pred_set))) and (label == 1):
                self.second_endpoint_to_S[i_edge] = True
        # don't forget to count degree :D
        if self.vertex_name in e and label==1:
            self.vertex_degree += 1
                
    def determine_cost(self):
        outside_edge_cost = 0
        if self.vertex_degree<=self.delta*self.n_vertex:
            return self.delta*self.n_vertex
        else:
            for i_edge in range(self.n_sample_edges):
                if ((self.first_endpoint_to_u[i_edge] and self.second_endpoint_to_S[i_edge]
                    ) or (self.second_endpoint_to_u[i_edge] and self.first_endpoint_to_S[i_edge])
                   ) and (self.edge_label[i_edge]==1):
                    outside_edge_cost +=1
            
            return outside_edge_cost*self.n_edges/self.n_sample_edges

In [28]:
class PredAwareUncluster:
    '''
    This is the class that implements the predecessor-aware non-edge sketh
    '''
    def __init__(self, vertex_name, n_vertex, pred_set, eps=0.2, delta=0.1):
        '''
        :param n_vertex: the the number of vertices in the graph
        '''
        if vertex_name in pred_set:
            raise ValueError('The tested vertex cannot be inside the set!')
        self.n_vertex = n_vertex
        self.n_edges = (int)(n_vertex*(n_vertex-1)/2)
        self.vertex_name = vertex_name
        self.vertex_degree = 0
        self.eps = eps
        self.delta = delta
        # sample a set of log n vertices
        self.n_sample_edges = max((int)(5*np.log(self.n_vertex)), 20)
        self.sample_edge_set = []
        self.first_endpoint_set = []
        self.second_endpoint_set = []
        # keep track of the endpoints' connection to u and S
        self.first_endpoint_to_u = self.n_sample_edges*[False]
        self.first_endpoint_to_S = self.n_sample_edges*[False]
        self.second_endpoint_to_u = self.n_sample_edges*[False]
        self.second_endpoint_to_S = self.n_sample_edges*[False]
        for i_edge in range(self.n_sample_edges):
            # to avoid sample the same vertex, try repetitive sampling
            while(True):
                first_vertex = np.random.choice(self.n_vertex, 1)[0]
                second_vertex = np.random.choice(self.n_vertex, 1)[0]
                if first_vertex!=second_vertex:
                    break
            self.sample_edge_set.append((first_vertex, second_vertex))
            self.first_endpoint_set.append(first_vertex)
            self.second_endpoint_set.append(second_vertex)
            if first_vertex == self.vertex_name:
                self.first_endpoint_to_u[i_edge] = True
            if second_vertex == self.vertex_name:
                self.second_endpoint_to_u[i_edge] = True
            if first_vertex in pred_set:
                self.first_endpoint_to_S[i_edge] = True
            if second_vertex in pred_set:
                self.second_endpoint_to_S[i_edge] = True
        # edge labels
        self.edge_label = self.n_sample_edges*[0]
        self.pred_set = pred_set
        
    def read_edge_and_process(self, e, label):
        for i_edge in range(self.n_sample_edges):
            # if edge is some (x,y), update the label
            this_edge_name = self.sample_edge_set[i_edge]
            if this_edge_name==e or this_edge_name==(e[1],e[0]):
                self.edge_label[i_edge] = label
            # if edge connects x (or y) to u, mark it
            if (self.first_endpoint_set[i_edge] in e) and (self.vertex_name in e) and (label == 1):
                self.first_endpoint_to_u[i_edge] = True
            if (self.second_endpoint_set[i_edge] in e) and (self.vertex_name in e) and (label == 1):
                self.second_endpoint_to_u[i_edge] = True
            # if there is intersection with S, mark it
            if (self.first_endpoint_set[i_edge] in e) and list(set(e).
                                                               intersection(set(self.pred_set))) and (label == 1):
                self.first_endpoint_to_S[i_edge] = True
            if (self.second_endpoint_set[i_edge] in e) and list(set(e).intersection(set(self.pred_set))) and (label == 1):
                self.second_endpoint_to_S[i_edge] = True
        # don't forget to count degree :D
        if self.vertex_name in e and label==1:
            self.vertex_degree += 1
                
    def determine_cost(self):
        outside_uncluster_cost = 0
        if self.vertex_degree<=self.delta*self.n_vertex:
            return self.delta*self.n_vertex
        for i_edge in range(self.n_sample_edges):
            if (not self.first_endpoint_to_S[i_edge]
               ) and (not self.second_endpoint_to_S[i_edge]
                     ) and self.edge_label[i_edge]==1:
                if (self.first_endpoint_to_u[i_edge]) and (not self.second_endpoint_to_S[i_edge]):
                    outside_uncluster_cost +=1
                if (not self.first_endpoint_to_u[i_edge]) and (self.second_endpoint_to_S[i_edge]):
                    outside_uncluster_cost += 1
            
        return outside_uncluster_cost*self.n_edges/self.n_sample_edges

In [29]:
class GreedyMIS:
    '''
    This class stored all the edges between the sampled vertices and compute the greedy MIS in the end
    '''
    def __init__(self, ordered_set):
        '''
        :param ordered_set: an ordered set of vertices
        '''
        # cast as array if not already
        self.num_stored_edge = 0
        self.ordered_set = np.array(ordered_set)
        self.n_sampled_vertex = self.ordered_set.shape[0]
        self.neighbor_vertex_list = []
        self.neighbor_vertex_ind_list = []
        for i_vertex in range(self.n_sampled_vertex):
            self.neighbor_vertex_list.append([self.ordered_set[i_vertex]])
            self.neighbor_vertex_ind_list.append([i_vertex])
        
    def read_edge_and_process(self, e, label):
        if (label==1) and (e[0] in self.ordered_set) and (e[1] in self.ordered_set):
            vertex_ind_one = np.where(self.ordered_set==e[0])[0][0]
            vertex_ind_two = np.where(self.ordered_set==e[1])[0][0]
            center_ind = min(vertex_ind_one, vertex_ind_two)
            neighbor_ind = max(vertex_ind_one, vertex_ind_two)
            self.neighbor_vertex_list[center_ind].append(self.ordered_set[neighbor_ind])
            self.neighbor_vertex_ind_list[center_ind].append(neighbor_ind)
            self.num_stored_edge += self.num_stored_edge
        else:
            pass
        
    def computeMIS(self):
        MIS_lists = []
        available_inds = np.arange(self.n_sampled_vertex).tolist()
        while len(available_inds)>0:
            remain_vertices = self.ordered_set[np.array(available_inds)]
            current_center_ind = available_inds[0]
            current_component = set(self.neighbor_vertex_list[current_center_ind]).intersection(set(remain_vertices))
            MIS_lists.append(list(current_component))
            available_inds = list(set(available_inds)-set(self.neighbor_vertex_ind_list[current_center_ind]))
        
        return MIS_lists

In [30]:
sbm_graph_stream.reset_index()
ordered_set = np.random.choice(sbm_graph_stream.n_vertex, 10, replace=False)
greedy_MIS_extracter = GreedyMIS(ordered_set)
for i in tqdm.tqdm(range(sbm_graph_stream.num_edges)):
    this_edge, this_edge_label = sbm_graph_stream.read_next_edge()
    if this_edge is None:
        break
    greedy_MIS_extracter.read_edge_and_process(this_edge, this_edge_label)
test_MIS_comp = greedy_MIS_extracter.computeMIS()

100%|█████████▉| 499498/499500 [00:01<00:00, 292398.92it/s]


In [31]:
print(test_MIS_comp)
print(ordered_set)

[[873, 547, 606], [402], [720, 132, 29], [708], [645], [151]]
[547 402  29 606 645 132 873 151 720 708]


### In this first version of implementation allow two passes to bring down the memory

In [32]:
def pivot_alg_two_pass(graph_stream, eps=0.2, delta=0.1, add_error=False, rescale_spr=False):
    n_vertex = graph_stream.n_vertex
    n_edges = sbm_graph_stream.num_edges
    n_sample_vertex = min(13, int(1/delta)) # cap it at 13 since otherwise the memory explodes
    total_edges_store = 0
    
    # initialize all the tools for process
    print('Pre-processing...')
    ordered_set = np.random.choice(sbm_graph_stream.n_vertex, n_sample_vertex, replace=False)
    # greedy MIS extractor
    greedy_MIS_extracter = GreedyMIS(ordered_set)
    
    print('Processing first pass of the stream...')
    # read the stream and process on-the-fly
    for i in tqdm.tqdm(range(n_edges)):
        this_edge, this_edge_label = sbm_graph_stream.read_next_edge()
        if this_edge is None:
            break
        greedy_MIS_extracter.read_edge_and_process(this_edge, this_edge_label)
    total_edges_store = total_edges_store + greedy_MIS_extracter.num_stored_edge
    
    print('Transition between the passes...')
    greedy_MIS_comp = greedy_MIS_extracter.computeMIS()
    prev_set = np.array([])
    NE_tool_list = []
    E_tool_list = []
    Unclust_tool_list = []
    for i_comp in range(len(greedy_MIS_comp)):
        u_vertex = greedy_MIS_comp[i_comp][0]
        NE_tool_list.append(PredAwareNE(u_vertex, n_vertex, prev_set))
        E_tool_list.append(PredAwareE(u_vertex, n_vertex, prev_set))
        Unclust_tool_list.append(PredAwareUncluster(u_vertex, n_vertex, prev_set))
        prev_set = np.array(prev_set.tolist()+greedy_MIS_comp[i_comp])
        # count the number of vertex pairs used by the tool
        total_edges_store += NE_tool_list[i_comp].n_sample_edges
        total_edges_store += E_tool_list[i_comp].n_sample_edges
        total_edges_store += Unclust_tool_list[i_comp].n_sample_edges
      
        
    # this line is the 'cheating' that we allow a second pass
    graph_stream.reset_index()
    
    print('Processing second pass of the stream...')
    # read the stream and process on-the-fly
    for i in tqdm.tqdm(range(n_edges)):
        this_edge, this_edge_label = sbm_graph_stream.read_next_edge()
        if this_edge is None:
            break
        for i_comp in range(len(greedy_MIS_comp)):
            NE_tool_list[i_comp].read_edge_and_process(this_edge, this_edge_label)
            E_tool_list[i_comp].read_edge_and_process(this_edge, this_edge_label)
            Unclust_tool_list[i_comp].read_edge_and_process(this_edge, this_edge_label)
    
    # post-processing
    print('Post-processing...')
    total_cost = 0
    for i_comp in range(len(greedy_MIS_comp)):
        total_cost = total_cost + NE_tool_list[i_comp].determine_cost()
        total_cost = total_cost + E_tool_list[i_comp].determine_cost()
        total_cost = total_cost + Unclust_tool_list[i_comp].determine_cost()
    
    return total_cost, total_edges_store

In [33]:
sbm_graph_stream.reset_index()
pivot_cost_est, pivot_total_stored_edge = pivot_alg_two_pass(sbm_graph_stream)

  9%|▊         | 42632/499500 [00:00<00:01, 426277.72it/s]

Pre-processing...
Processing first pass of the stream...


  0%|          | 641/499500 [00:00<02:35, 3216.92it/s]

Transition between the passes...
Processing second pass of the stream...


100%|█████████▉| 499498/499500 [02:36<00:00, 3197.93it/s]

Post-processing...


In [34]:
print('The expected SBM cost is ', sbm_graph_stream.cc_cost)
print('The estimated correlation clustering cost is ', pivot_cost_est)

The expected SBM cost is  100041
The estimated correlation clustering cost is  176294.11764705883


In [35]:
print('The number of edges in the graphs is ', sbm_graph_stream.num_edges)
print('The number of edges stored by the algorithm is ', pivot_total_stored_edge)

The number of edges in the graphs is  499500
The number of edges stored by the algorithm is  306
